In [1]:
LANDING = "https://en.wiktionary.org/wiki/Category:Hungarian_terms_with_audio_links"

In [3]:
!pip install bs4

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)


In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
rq = requests.get(LANDING)

In [6]:
assert rq.status_code == 200

In [7]:
rq.text

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Category:Hungarian terms with audio links - Wiktionary, the free dictionary</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"886875b4-c049-4966-95b2-12110eac84e5","wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":14,"wgPageName":"Category:Hungarian_terms_with_audio_links","wgTitle":"Hungarian terms with audio links","wgCurRevisionId":47076145,"wgRevisionId":47076145,"wgArticleId":4376193,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages using DynamicPageList","Hungarian entry maintenance","Terms

In [8]:
soup = BeautifulSoup(rq.text)

In [18]:
cat = soup.find("div", {"class": "mw-category-generated"})

In [ ]:
cat.findAll('li')

In [26]:
def get_category(soup: BeautifulSoup):
    links = []
    cat = soup.find("div", {"class": "mw-category-generated"})
    for link in cat.findAll('li'):
        links.append(f"https://en.wiktionary.org{link.find('a')['href']}")
    return links

In [33]:
def get_next_link(soup: BeautifulSoup):
    cat = soup.find("div", {"class": "mw-category-generated"})
    for a in cat.findAll('a'):
        if a.text.strip() == 'next page':
            return f"https://en.wiktionary.org{a['href']}"
    return None

In [ ]:
import time
links = []
next_page = LANDING
while next_page is not None:
    req = requests.get(next_page)
    soup = BeautifulSoup(req.text)
    links += get_category(soup)
    next_page = get_next_link(soup)
    time.sleep(3)
    

In [55]:
with open("wiktionary-audio-links.txt", "w") as of:
    for link in links:
        of.write(link + "\n")

In [56]:
req = requests.get(links[0])

In [57]:
req.text

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>-a - Wiktionary, the free dictionary</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"5d8e31b2-7632-4515-91cb-a90a0b80c082","wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"-a","wgTitle":"-a","wgCurRevisionId":78229049,"wgRevisionId":78229049,"wgArticleId":51875,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages with raw sortkeys","English terms with redundant sortkeys","English entries with language name categories using raw markup","Quotation templates to be cleaned","Requests for etym

In [58]:
bs = BeautifulSoup(req.text)

In [91]:
req = requests.get("https://en.wiktionary.org/wiki/apa")
bs = BeautifulSoup(req.text)

In [95]:
def extract_language_chunk(soup: BeautifulSoup, needle: str):
    pieces = []
    reading = False

    content = soup.find('div', {"id": "mw-content-text"})
    body_content = soup.find('div', {"id", "mw-parser-output"})

    for child in body_content.children:
        if child.name == "h2":
            if not reading and needle in child.text:
                reading = True
            if reading and not needle in child.text:
                reading = False
        else:
            if reading:
                pieces.append(child)

    return ''.join([str(x) for x in pieces])


In [98]:
sub = extract_language_chunk(bs, "Hungarian")

In [101]:
subsoup = BeautifulSoup(sub)

In [112]:
def get_ipa(subsoup: BeautifulSoup):
    ipa = []
    for pron in subsoup.findAll("span", {"class": "IPA"}):
        ipa.append(pron.text)
    filtered = []
    if len(ipa) > 1:
        for pron in ipa:
            if pron.startswith("-"):
                continue
            else:
                filtered.append(pron)
    return filtered

In [113]:
ipa = get_ipa(subsoup)
ipa

['[ˈɒpɒ]']

In [114]:
def get_audio_links(subsoup: BeautifulSoup):
    audio = []
    for pron in subsoup.findAll("audio"):
        for filesrc in pron.findAll("source"):
            src = filesrc["src"]
            if src.startswith("//"):
                src = "https" + src
            if "/transcoded/" in src:
                continue
            else:
                audio.append(src)
    return audio

In [118]:
def get_audio_from_page(page: str, language: str):
    pieces = page.split("/")
    word = pieces[-1]

    req = requests.get(page)
    assert req.status_code == 200

    soup = BeautifulSoup(req.text)

    html_chunk = extract_language_chunk(soup, language)
    subsoup = BeautifulSoup(html_chunk)

    ipa = get_ipa(subsoup)
    audio = get_audio_links(subsoup)

    return {
        "word": word,
        "ipa": ipa,
        "audio": audio
    }

    

In [121]:
from time import sleep
audio_parts = []
for link in links:
    audio_parts.append(get_audio_from_page(link, "Hungarian"))
    sleep(1)

ConnectionError: HTTPSConnectionPool(host='en.wiktionary.org', port=443): Max retries exceeded with url: /wiki/var (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fa427acb220>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

In [123]:
import json
with open("wiktionary_audio.json", "w") as of:
    json.dump(audio_parts, of)

In [133]:
def proc_ipa(ipa_string):
    if ipa_string.startswith('[') and ipa_string.endswith(']'):
        ipa_string = ipa_string[1:-1]
    ipa_string = ipa_string.replace("ˈ", "")
    expanded = " ".join(list(ipa_string))
    expanded = expanded.replace(" ː", "ː").replace(" ͡ ", "͡")
    expanded = expanded.replace("ɱ", "m")
    expanded = expanded.replace(" u ̯",  "u̯")
    expanded = expanded.replace("ˌ", "")
    return expanded

In [131]:
from urllib.parse import unquote

pronunciations = []

for item in audio_parts:
    word = item["word"]
    for ipa in item["ipa"]:
        pronunciations.append(f"{word}\t{proc_ipa(ipa)}")


In [132]:
with open("hungarian_wiktionary.dict", "w") as of:
    for pr in pronunciations:
        of.write(pr + "\n")

In [134]:
import os
os.mkdir("/home/joregan/wiktionary_hu")

In [143]:
with open("grabber.sh", "w") as grab:
    for item in audio_parts:
        word = unquote(item["word"])
        i = 0
        for i in range(0, len(item["audio"])):
            num = str(i) if i != 0 else ""
            ext = item['audio'][i].split(".")[-1]
            if item['ipa'] == []:
                comment = "# "
            else:
                comment = ""
            grab.write(f"{comment}wget {item['audio'][i].replace('https//', 'https://')} -O /home/joregan/wiktionary_hu/{word}{num}.{ext}\n")
            grab.write(f"{comment}sleep 5\n")
            with open(f"/home/joregan/wiktionary_hu/{word}{num}.txt", "w") as of:
                of.write(word.replace("_", " "))


In [1]:
def proc_ipa_base(ipa_string):
    if ipa_string.startswith('[') and ipa_string.endswith(']'):
        ipa_string = ipa_string[1:-1]
    elif ipa_string.startswith('/') and ipa_string.endswith('/'):
        ipa_string = ipa_string[1:-1]
    ipa_string = ipa_string.replace("ˈ", "")
    ipa_string = ipa_string.replace("ˌ", "")
    return ipa_string


def proc_ipa_kashubian(ipa_string):
    ipa_string = proc_ipa_base(ipa_string)
    ipa_string = ipa_string.replace("ʒk", "ʃk")
    ipa_string = ipa_string.replace("ã", "ã")
    ipa_string = ipa_string.replace("õ", "õ")
    ipa_string = ipa_string.replace("̊", "")
    ipa_string = ipa_string.replace("ʂ", "ʃ")
    ipa_string = ipa_string.replace("ʐ", "ʒ")
    expanded = " ".join(list(ipa_string))
    expanded = expanded.replace(" ː", "ː").replace(" ͡ ", "͡")
    expanded = expanded.replace(" ʲ", "ʲ")
    expanded = expanded.replace(" ̝", "̝")
    expanded = expanded.replace("ɱ", "m")
    expanded = expanded.replace(" u ̯",  "u̯")
    return expanded


In [6]:
words = {}
with open("csb.dict") as f:
    for line in f.readlines():
        if line.startswith("-"):
            continue
        word, pron = line.strip().split("\t")
        word = word.lower()
        pron = proc_ipa_kashubian(pron)
        if word.endswith("-"):
            continue
        if not word in words:
            words[word] = []
        words[word].append(pron)
with open("wiktionary-kashubian.dict", "w") as of:
    for word in words:
        for pron in words[word]:
            of.write(f"{word}\t{pron}\n")